In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA(GPU)를 사용합니다.")
else:
    device = torch.device("cpu")
    print("CUDA(GPU)를 사용할 수 없으므로, CPU를 사용합니다.")

CUDA(GPU)를 사용합니다.


In [3]:
import numpy as np
from torchvision import datasets, transforms
transform2 = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])
# 데이터셋 불러오기
nespresso_dataset = datasets.ImageFolder(root="./garbage_classification", transform=transform2)
# DataLoader를 사용하여 이미지 데이터를 배치 단위로 불러올 수 있음
nespresso_data_loader = torch.utils.data.DataLoader(nespresso_dataset, batch_size=1, shuffle=False)
# 이미지 데이터를 NumPy 배열로 변환하고 평균과 표준편차를 계산
pixel_values = []
for images, _ in nespresso_data_loader:
    pixel_values.append(images.numpy())
pixel_values = np.concatenate(pixel_values, axis=0)
# 픽셀 값의 평균과 표준편차 계산
mean = np.mean(pixel_values, axis=(0, 2, 3))
std = np.std(pixel_values, axis=(0, 2, 3))
print("평균:", mean)
print("표준편차:", std)

평균: [0.658039   0.61618674 0.58561134]
표준편차: [0.2715358  0.28308678 0.2974301 ]


In [3]:
class myResNet(nn.Module):
    def __init__(self):
        super(myResNet, self).__init__()
        # ResNet18 모델 로드
        self.resnet = models.resnet18(weights=None)
        # self.resnet = models.resnet50(pretrained=False)
        # 출력 레이어를 클래스 수에 맞게 조정
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 12)

    def forward(self, x):
        return self.resnet(x)

In [3]:
class myVggNet(nn.Module):
    def __init__(self):
        super(myVggNet, self).__init__()
        # vgg16 모델 로드
        self.vggnet = models.vgg16()
        # 출력 레이어를 클래스 수에 맞게 조정
        num_ftrs = self.vggnet.classifier[6].in_features
        self.vggnet.classifier[6].in_features = nn.Linear(num_ftrs, 12)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.vggnet.features(x)
        x = self.vggnet.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.vggnet.classifier(x)
        x = self.dropout(x)
        return x

In [4]:
class myVggNet(nn.Module):
    def __init__(self):
        super(myVggNet, self).__init__()
        # vgg16 모델 로드
        self.vggnet = models.vgg16()
        # 출력 레이어를 클래스 수에 맞게 조정
        num_ftrs = self.vggnet.classifier[6].in_features
        self.vggnet.classifier[6].in_features = nn.Linear(num_ftrs, 12)

    def forward(self, x):
        return self.vggnet(x)

평균: [0.658039   0.61618674 0.58561134]
표준편차: [0.2715358  0.28308678 0.2974301 ]

In [5]:
# CIFAR10 데이터셋 로드 및 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((64, 64)),
    transforms.Normalize([0.6580, 0.6161, 0.5856], [0.2715, 0.2830, 0.2974]) # 알려진 CIFAR10의 평균과 표준편차
])

# 데이터셋 불러오기
garbage_dataset = datasets.ImageFolder(root="./garbage_classification", transform=transform)
# DataLoader를 사용하여 이미지 데이터를 배치 단위로 불러올 수 있음
garbage_loader = torch.utils.data.DataLoader(garbage_dataset, batch_size=1, shuffle=False)

total_size = len(garbage_dataset)

# 훈련 세트의 비율 설정
train_ratio = 0.8

# 훈련 세트와 테스트 세트로 나누기
train_size = int(train_ratio * total_size)
test_size = total_size - train_size
train_dataset, test_dataset = torch.utils.data.random_split(garbage_dataset, [train_size, test_size])

# 훈련과 테스트 데이터로더 생성
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [5]:
# 모델, 손실 함수, 최적화 알고리즘 설정
model = myResNet().to(device)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.5)
optimizer = optim.Adam(model.parameters(), lr=0.00005)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [6]:
model = myVggNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters(), lr=0.001)

In [5]:
model = myResNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters(), lr=0.001)

In [ ]:
model = myVggNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(lr=0.0001)

In [6]:
# 모델, 손실 함수, 최적화 알고리즘 설정
model = myVggNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.7)
# optimizer = optim.Adam(model.parameters(), lr=0.0001)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [7]:
# 손실과 정확도를 저장하기 위한 리스트 초기화
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

In [8]:
# 훈련 함수    
def train(model, train_loader, optimizer, epoch):

    
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for data, target in train_loader:
        if torch.cuda.is_available():
            data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    # 예측값 계산(가장 높은 값을 가진 인덱스)
    _, predicted = torch.max(output.data, 1)
    total += target.size(0)
    # 올바르게 예측된 샘플의 수 업데이트
    correct += (predicted == target).sum().item()

    # 평균 훈련 손실 계산
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    print(f'Epoch {epoch}, Training loss: {train_loss:.4f}')
    accuracy = 100. * correct / total
    train_accuracies.append(accuracy)

In [9]:
# 테스트 함수
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            if torch.cuda.is_available():
                data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)
    test_losses.append(test_loss)
    test_accuracies.append(accuracy)
    print(f'Test loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%')

In [10]:
# batchsize 256, 128
# batch normalization 추가
# vggnet16
# optimizer: adagrad(lr: 0.001)
# 훈련 및 테스트 실행
for epoch in range(1, 21):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

c:\Users\lma00\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1, Training loss: 49.2805
Test loss: 2.0250, Accuracy: 37.35%
Epoch 2, Training loss: 1.9633
Test loss: 1.7854, Accuracy: 41.70%
Epoch 3, Training loss: 1.8063
Test loss: 1.7911, Accuracy: 41.48%
Epoch 4, Training loss: 1.6389
Test loss: 1.4777, Accuracy: 50.08%
Epoch 5, Training loss: 1.4431
Test loss: 1.3106, Accuracy: 56.17%
Epoch 6, Training loss: 1.3015
Test loss: 1.2408, Accuracy: 57.27%
Epoch 7, Training loss: 1.1720
Test loss: 1.1540, Accuracy: 60.23%
Epoch 8, Training loss: 1.1048
Test loss: 1.1696, Accuracy: 59.68%
Epoch 9, Training loss: 1.0130
Test loss: 1.0284, Accuracy: 66.48%
Epoch 10, Training loss: 0.9733
Test loss: 1.0671, Accuracy: 64.65%
Epoch 11, Training loss: 0.9027
Test loss: 0.8895, Accuracy: 70.35%
Epoch 12, Training loss: 0.8446
Test loss: 0.9197, Accuracy: 68.32%
Epoch 13, Training loss: 0.7935
Test loss: 1.1083, Accuracy: 68.22%
Epoch 14, Training loss: 0.7601
Test loss: 0.9719, Accuracy: 66.87%
Epoch 15, Training loss: 0.7165
Test loss: 0.8719, Accur

In [9]:
# batchsize 256, 128
# batch normalization 추가
# resnet
# optimizer: adagrad(lr: 0.001)
# 훈련 및 테스트 실행
for epoch in range(1, 21):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

c:\Users\lma00\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1, Training loss: 1.2023
Test loss: 1.0285, Accuracy: 66.19%
Epoch 2, Training loss: 0.6222
Test loss: 0.9156, Accuracy: 69.48%
Epoch 3, Training loss: 0.2880
Test loss: 0.8694, Accuracy: 72.38%
Epoch 4, Training loss: 0.1138
Test loss: 0.9151, Accuracy: 72.61%
Epoch 5, Training loss: 0.0487
Test loss: 0.8865, Accuracy: 73.70%
Epoch 6, Training loss: 0.0266
Test loss: 0.9140, Accuracy: 73.38%
Epoch 7, Training loss: 0.0165
Test loss: 0.9121, Accuracy: 73.70%
Epoch 8, Training loss: 0.0123
Test loss: 0.9227, Accuracy: 73.90%
Epoch 9, Training loss: 0.0099
Test loss: 0.9346, Accuracy: 74.22%
Epoch 10, Training loss: 0.0085
Test loss: 0.9377, Accuracy: 74.09%
Epoch 11, Training loss: 0.0073
Test loss: 0.9497, Accuracy: 73.93%
Epoch 12, Training loss: 0.0064
Test loss: 0.9524, Accuracy: 74.32%
Epoch 13, Training loss: 0.0057
Test loss: 0.9582, Accuracy: 74.38%
Epoch 14, Training loss: 0.0061
Test loss: 0.9642, Accuracy: 73.99%
Epoch 15, Training loss: 0.0049
Test loss: 0.9745, Accura

In [11]:
# batchsize 256, 128
# batch normalization 추가
# resnet
# optimizer: adagrad(lr: 0.0001)
# 훈련 및 테스트 실행
for epoch in range(1, 21):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

c:\Users\lma00\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1, Training loss: 1.6747
Test loss: 1.4940, Accuracy: 52.14%
Epoch 2, Training loss: 1.2991
Test loss: 1.3336, Accuracy: 57.56%
Epoch 3, Training loss: 1.1517
Test loss: 1.2476, Accuracy: 60.43%
Epoch 4, Training loss: 1.0556
Test loss: 1.1905, Accuracy: 62.68%
Epoch 5, Training loss: 0.9828
Test loss: 1.1495, Accuracy: 63.33%
Epoch 6, Training loss: 0.9220
Test loss: 1.1174, Accuracy: 64.13%
Epoch 7, Training loss: 0.8709
Test loss: 1.0942, Accuracy: 64.81%
Epoch 8, Training loss: 0.8258
Test loss: 1.0753, Accuracy: 65.32%
Epoch 9, Training loss: 0.7864
Test loss: 1.0575, Accuracy: 65.90%
Epoch 10, Training loss: 0.7500
Test loss: 1.0439, Accuracy: 66.71%
Epoch 11, Training loss: 0.7165
Test loss: 1.0338, Accuracy: 67.03%
Epoch 12, Training loss: 0.6863
Test loss: 1.0220, Accuracy: 67.16%
Epoch 13, Training loss: 0.6630
Test loss: 1.0148, Accuracy: 67.39%
Epoch 14, Training loss: 0.6310
Test loss: 1.0081, Accuracy: 67.68%
Epoch 15, Training loss: 0.6060
Test loss: 0.9988, Accura

In [10]:
# batchsize 256, 128
# batch normalization 추가
# vgg16
# optimizer: sgd(momentum:0.7, lr: 0.0001), dropout
# 훈련 및 테스트 실행
for epoch in range(1, 21):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

c:\Users\lma00\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


RuntimeError: running_mean should contain 256 elements not 512

In [9]:
# batchsize 256, 128
# vgg16
# optimizer: sgd(momentum:0.7, lr: 0.0001), dropout
# 훈련 및 테스트 실행
for epoch in range(1, 21):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

c:\Users\lma00\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1, Training loss: 6.8377
Test loss: 6.6787, Accuracy: 33.77%
Epoch 2, Training loss: 6.3552
Test loss: 5.5375, Accuracy: 33.77%
Epoch 3, Training loss: 5.6037
Test loss: 4.8782, Accuracy: 33.77%
Epoch 4, Training loss: 5.3675
Test loss: 4.6688, Accuracy: 33.77%


KeyboardInterrupt: 

In [9]:
# batchsize 256, 128
# vgg19
# optimizer: sgd(momentum:0.7, lr: 0.0001)
# 훈련 및 테스트 실행
for epoch in range(1, 21):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

c:\Users\lma00\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1, Training loss: 6.8507
Test loss: 6.7872, Accuracy: 34.06%
Epoch 2, Training loss: 6.6730
Test loss: 6.4357, Accuracy: 34.06%
Epoch 3, Training loss: 5.3104
Test loss: 3.7853, Accuracy: 34.06%
Epoch 4, Training loss: 3.2074
Test loss: 2.4229, Accuracy: 34.06%
Epoch 5, Training loss: 2.7506
Test loss: 2.3572, Accuracy: 34.06%
Epoch 6, Training loss: 2.6503
Test loss: 2.3007, Accuracy: 34.06%
Epoch 7, Training loss: 2.5734
Test loss: 2.2800, Accuracy: 34.06%


KeyboardInterrupt: 

In [30]:
# batchsize 256, 128
# vgg19
# optimizer: sgd(momentum:0.3, lr: 0.01)
# 훈련 및 테스트 실행
for epoch in range(1, 21):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

Epoch 1, Training loss: 6.5654
Test loss: 6.6787, Accuracy: 5.45%
Epoch 2, Training loss: 3.8817
Test loss: 2.3020, Accuracy: 12.25%
Epoch 3, Training loss: 2.2478
Test loss: 2.1796, Accuracy: 34.87%
Epoch 4, Training loss: 2.1996
Test loss: 2.0568, Accuracy: 34.87%
Epoch 5, Training loss: 2.2119
Test loss: 2.4389, Accuracy: 12.25%
Epoch 6, Training loss: 2.1611
Test loss: 1.9945, Accuracy: 34.87%
Epoch 7, Training loss: 2.1475
Test loss: 2.1833, Accuracy: 34.87%


KeyboardInterrupt: 

In [10]:
# batchsize 256, 128, lr: 0.001
# vgg19
# optimizer: sgd(momentum:0.5)
# 훈련 및 테스트 실행
for epoch in range(1, 21):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

c:\Users\lma00\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1, Training loss: 4.3165
Test loss: 2.5205, Accuracy: 34.58%
Epoch 2, Training loss: 2.4739
Test loss: 2.2039, Accuracy: 34.58%
Epoch 3, Training loss: 2.3432
Test loss: 2.1345, Accuracy: 34.71%
Epoch 4, Training loss: 2.2413
Test loss: 2.0539, Accuracy: 38.54%
Epoch 5, Training loss: 2.1392
Test loss: 1.9735, Accuracy: 40.35%


KeyboardInterrupt: 

In [10]:
# batchsize 256, 128, lr: 0.0001
# vgg19
# optimizer: adam
# 훈련 및 테스트 실행
for epoch in range(1, 21):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

c:\Users\lma00\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1, Training loss: 2.7572
Test loss: 1.9796, Accuracy: 34.55%
Epoch 2, Training loss: 1.8957
Test loss: 1.6729, Accuracy: 44.31%
Epoch 3, Training loss: 1.5410
Test loss: 1.3383, Accuracy: 51.79%
Epoch 4, Training loss: 1.3725
Test loss: 1.2343, Accuracy: 58.20%
Epoch 5, Training loss: 1.1464
Test loss: 1.1023, Accuracy: 62.58%
Epoch 6, Training loss: 1.0069
Test loss: 0.9684, Accuracy: 67.48%
Epoch 7, Training loss: 0.8956
Test loss: 0.9131, Accuracy: 69.87%
Epoch 8, Training loss: 0.8191
Test loss: 0.8862, Accuracy: 70.93%
Epoch 9, Training loss: 0.7222
Test loss: 0.8414, Accuracy: 74.64%
Epoch 10, Training loss: 0.6504
Test loss: 0.7660, Accuracy: 75.15%
Epoch 11, Training loss: 0.5856
Test loss: 0.7938, Accuracy: 74.99%
Epoch 12, Training loss: 0.5160
Test loss: 0.7527, Accuracy: 77.44%
Epoch 13, Training loss: 0.4630
Test loss: 0.8798, Accuracy: 76.67%
Epoch 14, Training loss: 0.4269
Test loss: 0.7597, Accuracy: 77.96%
Epoch 15, Training loss: 0.3308
Test loss: 0.7542, Accura

In [24]:
# batchsize 256, 128, lr: 0.00005
# vgg16
# 훈련 및 테스트 실행
for epoch in range(1, 21):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

c:\Users\lma00\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1, Training loss: 2.9062
Test loss: 2.0179, Accuracy: 39.90%
Epoch 2, Training loss: 1.9282
Test loss: 1.6411, Accuracy: 44.05%
Epoch 3, Training loss: 1.5399
Test loss: 1.3002, Accuracy: 56.72%
Epoch 4, Training loss: 1.2604
Test loss: 1.1391, Accuracy: 61.71%
Epoch 5, Training loss: 1.0905
Test loss: 1.0173, Accuracy: 65.97%
Epoch 6, Training loss: 1.0532
Test loss: 1.0016, Accuracy: 67.32%
Epoch 7, Training loss: 0.9187
Test loss: 0.8920, Accuracy: 70.13%
Epoch 8, Training loss: 0.8544
Test loss: 0.9046, Accuracy: 69.84%
Epoch 9, Training loss: 0.7871
Test loss: 0.7950, Accuracy: 73.54%
Epoch 10, Training loss: 0.6780
Test loss: 0.7853, Accuracy: 73.74%
Epoch 11, Training loss: 0.6462
Test loss: 0.8680, Accuracy: 72.22%
Epoch 12, Training loss: 0.6117
Test loss: 0.7917, Accuracy: 74.51%
Epoch 13, Training loss: 0.5419
Test loss: 0.7323, Accuracy: 75.73%
Epoch 14, Training loss: 0.4748
Test loss: 0.7391, Accuracy: 77.86%
Epoch 15, Training loss: 0.4412
Test loss: 0.8295, Accura

In [9]:
# batchsize 256, 128, lr: 0.00005
# 훈련 및 테스트 실행
for epoch in range(1, 21):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

c:\Users\lma00\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1, Training loss: 1.6411
Test loss: 1.2575, Accuracy: 58.56%
Epoch 2, Training loss: 1.0119
Test loss: 0.9980, Accuracy: 67.68%
Epoch 3, Training loss: 0.7274
Test loss: 0.9265, Accuracy: 70.09%
Epoch 4, Training loss: 0.4927
Test loss: 0.9135, Accuracy: 70.90%
Epoch 5, Training loss: 0.3022
Test loss: 0.9070, Accuracy: 71.35%
Epoch 6, Training loss: 0.1636
Test loss: 0.9253, Accuracy: 70.93%
Epoch 7, Training loss: 0.0858
Test loss: 0.9459, Accuracy: 71.77%
Epoch 8, Training loss: 0.0477
Test loss: 0.9693, Accuracy: 71.38%
Epoch 9, Training loss: 0.0305
Test loss: 0.9889, Accuracy: 71.25%
Epoch 10, Training loss: 0.0204
Test loss: 0.9941, Accuracy: 71.74%
Epoch 11, Training loss: 0.0173
Test loss: 1.0258, Accuracy: 71.64%
Epoch 12, Training loss: 0.0139
Test loss: 1.0248, Accuracy: 72.06%
Epoch 13, Training loss: 0.0113
Test loss: 1.0422, Accuracy: 71.38%
Epoch 14, Training loss: 0.0118


KeyboardInterrupt: 

In [27]:
# batchsize 256, 1000, lr: 0.00005
# 훈련 및 테스트 실행
for epoch in range(1, 21):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

Epoch 1, Training loss: 1.6483
Test loss: 1.2866, Accuracy: 56.11%
Epoch 2, Training loss: 1.0276
Test loss: 1.0378, Accuracy: 65.16%
Epoch 3, Training loss: 0.7291
Test loss: 0.9387, Accuracy: 69.09%
Epoch 4, Training loss: 0.4897
Test loss: 0.9124, Accuracy: 69.19%
Epoch 5, Training loss: 0.2932
Test loss: 0.9029, Accuracy: 69.84%
Epoch 6, Training loss: 0.1630
Test loss: 0.9217, Accuracy: 70.38%
Epoch 7, Training loss: 0.0832
Test loss: 0.9375, Accuracy: 70.32%
Epoch 8, Training loss: 0.0466
Test loss: 0.9611, Accuracy: 69.87%
Epoch 9, Training loss: 0.0299
Test loss: 0.9818, Accuracy: 70.13%


KeyboardInterrupt: 

In [16]:
# batchsize 128, 1000, lr: 0.00005
# 훈련 및 테스트 실행
for epoch in range(1, 21):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

c:\Users\lma00\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1, Training loss: 1.4699
Test loss: 1.1608, Accuracy: 61.10%
Epoch 2, Training loss: 0.8582
Test loss: 0.9826, Accuracy: 68.13%
Epoch 3, Training loss: 0.5489
Test loss: 0.9510, Accuracy: 69.22%
Epoch 4, Training loss: 0.3024
Test loss: 0.9674, Accuracy: 70.22%
Epoch 5, Training loss: 0.1355
Test loss: 0.9949, Accuracy: 70.45%
Epoch 6, Training loss: 0.0641
Test loss: 1.0083, Accuracy: 70.58%
Epoch 7, Training loss: 0.0384
Test loss: 1.0542, Accuracy: 70.32%
Epoch 8, Training loss: 0.0252
Test loss: 1.0635, Accuracy: 70.38%


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# 손실 및 정확도 시각화
fig, axs = plt.subplots(2, 1, figsize=(10, 10))

axs[0].plot(train_losses, label='Training Loss')
axs[0].plot(test_losses, label='Test Loss')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()
axs[0].set_title('Training and Test Loss')

axs[1].plot(train_accuracies, label='Training Accuracy', color='blue')
axs[1].plot(test_accuracies, label='Test Accuracy', color='orange')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy (%)')
axs[1].legend()
axs[1].set_title('Training and Test Accuracy')

plt.tight_layout()
plt.show()